In [ ]:
!pip install transformers=="4.0.0"

In [ ]:
import csv
import pandas as pd
from fastai.text.all import *
import torch
from transformers import GPT2TokenizerFast, GPT2LMHeadModel
import json

In [ ]:
def tokenize(text):
    toks = tokenizer.tokenize(text)
    return tensor(tokenizer.convert_tokens_to_ids(toks))

pretrained_weights = 'gpt2'
tokenizer = GPT2TokenizerFast.from_pretrained(pretrained_weights)

class TransformersTokenizer(Transform):
    def __init__(self, tokenizer): self.tokenizer = tokenizer
    def encodes(self, x): 
        return x if isinstance(x, Tensor) else tokenize(x)
        
    def decodes(self, x): return TitledStr(self.tokenizer.decode(x.cpu().numpy()))

class DropOutput(Callback):
    def after_pred(self):self.learn.pred=self.pred[0]


In [ ]:
artists={'Justin Bieber':'beiber',
        'Adele':'adele',
        'Drake':'drake',
        'The Weeknd':'weeknd',
        'Ariana Grande':'ariana',
        'Ed Sheeran':'edsheeran',
        'Coldplay':'coldplay',
        'Miley Cyrus':'miley',
        'Backstreet Boys':'backstreet',
        'Eminem':'eminem',
        '21 Savage':'21savage',
        'Linkin Park':'linkinpark',
        '\u200btwenty one pilots':'21pilots',
        'Kanye West':'kanye',
        'Taylor Swift':'taylor'}
dic={
    'Justin Bieber':['Love Yourself','Sorry','Baby','Purpose','Boyfriend','Nothing Like Us','Never Say Never'],
    'Adele':['Hello','Someone Like You','All I Ask','When We Were Young','Rolling in the Deep','Skyfall','Daydreamer'] ,
    'Drake':['God’s Plan','In My Feelings','Hotline Bling','One Dance','Nice For What','Hold On, We’re Going Home','Die In Your Arms'] ,
    'The Weeknd':['Starboy','Can’t Feel My Face','Call Out My Name','Reminder','Often','Feel It Coming','Wicked Games'] ,
    'Ariana Grande':['thank u, next','7 rings','God is a woman','no tears left to cry','Dangerous Woman','breathin','Bad Decisions'] ,
    'Ed Sheeran':['Shape of You','Perfect','Castle on the Hill','Happier','Photograph','Galway Girl','Nina'] ,
    'Coldplay':['The Scientist','Viva La Vida','Kaleidoscope','Yellow','Hymn for the Weekend','A Sky Full of Stars','Hypnotised'] ,
    'Miley Cyrus':['Malibu','Wrecking Ball','We Can’t Stop','Slide Away','Mother’s Daughter','Adore You','Party in the U.S.A.'] ,
    'Backstreet Boys':['I Want It That Way','Don’t Go Breaking My Heart','As Long As You Love Me','Everybody (Backstreet’s Back)','I’ll Never Break Your Heart','What Makes You Different (Makes You Beautiful)','OK'] ,
    'Eminem':['Rap God','The Monster','Lose Yourself','Lucky You','Stan','Love the Way You Lie','The Real Slim Shady'] ,
    '21 Savage':['Bank Account','a lot','Red Opps','ball w/o you','can’t leave without it','monster','Numb'] ,
    'Linkin Park':['Heavy','In the End','One More Light','Numb','Battle Symphony','Good Goodbye','CASTLE OF GLASS'] ,
    '\u200btwenty one pilots':['Heathens','Stressed Out','Ride','Chlorine','Nico and the Niners','Car Radio','Tear in My Heart'] ,
    'Kanye West':['Mercy','Father Stretch My Hands, Pt. 1','Ultralight Beam','Monster','West Bound 2','New Slaves','Blood on the Leaves'],
    'Taylor Swift':['Shake It Off','Blank Space','Bad Blood','Look What You Made Me Do','Gorgeous','Style']}

In [ ]:
song=[]
for i in dic:
    for k in dic[i]:
        song.append(k)


In [ ]:
with open('/kaggle/input/d/adityasrinivasmenon/lyrics-models/genius.csv',encoding="utf8") as f:
    reader = csv.reader(f)
    data = list(reader)
data[7]

In [ ]:
def batch(lyrics):
    prompt=''
    if lyrics[0]=='\n':
        count=-1
    else: count=0
    for i in range(len(lyrics)):
        prompt+=lyrics[i]
        if lyrics[i-1]=='\n' and lyrics[i]=='\n':
            
            continue
        if lyrics[i]=='\n':
            count+=1
        if count==4:
            break
    return prompt.replace('\n\n','\n')

def countns(lyrics):
    count=0
    for i in range(len(lyrics)):
        if lyrics[i]=='\n':
            count+=1
    return count

def linebreak(lyrics):
    for i in range(len(lyrics)):
        if lyrics[i]=='\n':
            return i+1
        
def extract_prompt(lyrics):
    if lyrics[0]=='\n':
        count=-1
    else: count=0
    for i in range(len(lyrics)):
        if lyrics[i]=='\n':
            count+=1
        if count==2:break
    return i+1
  
def extract_preds(lyrics):
    if lyrics[0]=='\n':
        count=-1
    else: count=0
    for i in range(len(lyrics)):
        if lyrics[i-1]=='\n' and lyrics[i]=='\n':
            
            continue
        if lyrics[i]=='\n':
            count+=1
        if count==5:break
    return i+1

In [ ]:
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

In [ ]:
dataset=[]

for k in data[1:]:
    if k[1] in song:
        dataset.append(k)
        prompts=[]
        generate=[]
        n=countns(k[2])
        string=k[2]
        learn=load_learner('/kaggle/input/d/adityasrinivasmenon/lyrics-models/{}.pkl'.format(artists[k[0]]))
        learn.model.to(device)
        print('/kaggle/input/d/adityasrinivasmenon/lyrics-models/{}.pkl'.format(artists[k[0]]))
        for j in range(n-4):
            try:
                p=batch(string)
                br=linebreak(p)
                string=string[br:]

                pr=p[:extract_prompt(p)]
                prompt_ids = tokenizer.encode(pr)
                inp = tensor(prompt_ids)[None].cuda()
                count=0
                while(True):
                    preds=learn.model.generate(inp,
                                 do_sample=True, 
                                 max_length=65, 
                                 min_length=5,
                                 top_k=40,
                                 num_return_sequences=1)

                    g=tokenizer.decode(preds[0].cpu().tolist())
                    g=g[:extract_preds(g)].replace("\"\"","").replace("\"","").replace("\n\n","\n").replace("\n\n","\n")
                    count+=1
                    if countns(g)>=4 or count==10:break


                generate.append(g)
                prompts.append(p)

            except IndexError:break
        dataset[dataset.index(k)].pop()       
        dataset[dataset.index(k)].append(prompts)
        dataset[dataset.index(k)].append(generate)
        
        
        del learn.model
        torch.cuda.empty_cache()
        
with open('lyrics.json', 'w') as json_file:
    json.dump(dataset, json_file)

In [ ]:
with open('/kaggle/working/lyrics.json') as f:
    data = json.load(f)